In [1]:
import os 

In [2]:
%pwd

'/home/prathameshdevadiga/personal/projs/KidneyDiseaseClassification/research'

In [3]:
os.chdir("../")

In [5]:
%pwd

'/home/prathameshdevadiga/personal/projs/KidneyDiseaseClassification'

In [6]:
from dataclasses import dataclass 
from pathlib import Path

@dataclass(frozen=True) 
class TrainingConfig: 
    root_dir: Path 
    trained_model_path: Path 
    updated_base_model_path: Path 
    training_data: Path  
    params_epochs: int 
    params_batch_size: int 
    params_is_augmentation: bool 
    params_image_size: int 

In [9]:
from cnnClassifier.constants import * 
from cnnClassifier.utils.common import read_yaml, create_directories
import tensorflow as tf

In [19]:
class ConfigurationManager: 
    def __init__ (
            self, 
            config_filepath = CONFIG_FILE_PATH, 
            params_filepath = PARAMS_FILE_PATH):
        
        self.config = read_yaml(config_filepath) 
        self.params = read_yaml(params_filepath) 

        create_directories([self.config.artifacts_root]) 


    def get_training_config(self) -> TrainingConfig: 
        training = self.config.training 
        prepare_base_model = self.config.prepare_base_model 
        params = self.params 
        training_data = os.path.join(self.config.data_ingestion.unzip_dir, "CT-KIDNEY-DATASET-Normal-Cyst-Tumor-Stone")
        create_directories([
            Path(training.root_dir) 
        ])

        training_config = TrainingConfig(
            root_dir=Path(training.root_dir), 
            trained_model_path=Path(training.trained_model_path),
            updated_base_model_path=Path(prepare_base_model.updated_base_model_path),
            training_data = Path(training_data),
            params_epochs=params.EPOCHS, 
            params_batch_size=params.BATCH_SIZE, 
            params_is_augmentation=params.AUGMENTATION, 
            params_image_size=params.IMAGE_SIZE
        )

        return training_config
    

In [20]:
import os 
import urllib.request as request 
from zipfile import ZipFile 
import tensorflow as tf 
import time 

In [21]:
class Training:
    def __init__(self, config: TrainingConfig):
        self.config = config 

    def get_base_model(self): 
        self.model = tf.keras.models.load_model(
            self.config.updated_base_model_path
        )

    # Not using train test split, rather using inbuild keras feature. 
        
    def train_valid_generator(self): 

        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split = 0.20
        )

        dataflow_kwargs = dict(
            target_size = self.config.params_image_size[:-1], 
            batch_size = self.config.params_batch_size,
            interpolation = "bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data, 
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )

        if self.config.params_is_augmentation:
            train_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
                rotation_range=40, 
                horizontal_flip=True, 
                width_shift_range=0.2,
                height_shift_range=0.2,
                shear_range=0.2,
                zoom_range=0.2,
                **datagenerator_kwargs
            )

        else:
            train_datagenerator = valid_datagenerator

        self.train_generator = train_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="training",
            shuffle=True,
            **dataflow_kwargs
        )


    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):
        model.save(path) 

    
    def train(self):
        self.steps_per_epoch = self.train_generator.samples // self.train_generator.batch_size
        self.validation_steps = self.valid_generator.samples // self.valid_generator.batch_size

        self.model.fit(
            self.train_generator,
            epochs = self.config.params_epochs, 
            steps_per_epoch = self.steps_per_epoch, 
            validation_steps = self.validation_steps, 
            validation_data = self.valid_generator, 
        )

        self.save_model(
            path = self.config.trained_model_path, 
            model = self.model
        )

In [22]:
try:
    config = ConfigurationManager() 
    training_config = config.get_training_config() 
    training = Training(config = training_config)
    training.get_base_model() 
    training.train_valid_generator() 
    training.train()

except Exception as e:
    raise e

[2024-01-02 09:53:22,822 : INFO: common: yaml file : config/config.yaml loaded successfully]
[2024-01-02 09:53:22,827 : INFO: common: yaml file : params.yaml loaded successfully]
[2024-01-02 09:53:22,828 : INFO: common: Created directory at : artifacts]
[2024-01-02 09:53:22,829 : INFO: common: Created directory at : artifacts/training]
Found 2489 images belonging to 1 classes.
Found 9957 images belonging to 1 classes.
Epoch 1/300


2024-01-02 09:53:24.178700: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]
2024-01-02 09:53:25.167660: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 205520896 exceeds 10% of free system memory.
2024-01-02 09:53:25.284549: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 205520896 exceeds 10% of free system memory.
2024-01-02 09:53:25.929470: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 102760448 exceeds 10% of free system memory.
2024-01-02 09:53:26.165632: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 102760448 exceeds 10% of free system memory.


  1/622 [..............................] - ETA: 51:38 - loss: 6.2257 - accuracy: 0.0625

2024-01-02 09:53:29.205715: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 205520896 exceeds 10% of free system memory.


622/622 [==============================] - ETA: 0s - loss: 101.3581 - accuracy: 0.2495

2024-01-02 11:03:52.876359: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


KeyboardInterrupt: 